In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|default_exp converter.yolo

# Yolo converter.

> Convert dataset formats.

In [ ]:
#|export
from fastcore.all import *
import shutil
import polvo as pv

/home/lgvaz/git/polvo/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
class YOLO(pv.Visitor):
    def convert_dataset(self, save_dir, classes, train=None, valid=None, test=None, n_workers=defaults.cpus):
        save_dir = Path(save_dir)
        dirs = {}
        if train: 
            train_dir = dirs['train_dir'] = pv.mkdir(save_dir/'train')
            self.convert_records(train_dir, train, n_workers=n_workers)
        if valid:
            valid_dir = dirs['valid_dir'] = pv.mkdir(save_dir/'valid')
            self.convert_records(valid_dir, valid, n_workers=n_workers)
        if test: 
            test_dir = dirs['test_dir'] = pv.mkdir(save_dir/'test')
            self.convert_records(test_dir, test, n_workers=n_workers)
            
        yolo_yaml = self.yolo_yaml(classes, save_dir, **dirs)
        pv.save_txt(yolo_yaml, save_dir/'dataset.yaml')
        
    def convert_records(self, save_dir, records, n_workers=defaults.cpus):
        image_dir = pv.mkdir(Path(save_dir)/'images')
        ann_dir = pv.mkdir(Path(save_dir)/'labels')
        parallel(pv.partial(self.convert_record, image_dir=image_dir, ann_dir=ann_dir),
                 records, progress=pv.pbar, n_workers=n_workers)
        
    def convert_record(self, record, image_dir, ann_dir):
        self._labels, self._bboxes = [], []
        self.visit_all(record)
        lines = [' '.join(o) for o in pv.safe_zip(self._labels, self._bboxes)]
        pv.save_txt('\n'.join(lines), ann_dir/self._image_file.with_suffix('.txt').name)
        shutil.copy(self._image_file.absolute(), image_dir/self._image_file.name)
        
    def yolo_yaml(self, class_map, save_dir, train_dir=None, valid_dir=None, test_dir=None):
        classes = '\n'.join([f'  {k}: {v}' for k,v in class_map.items()])
        train_str = f"train: {train_dir.relative_to(save_dir)}\n" if train_dir is not None else ""
        valid_str = f"val: {valid_dir.relative_to(save_dir)}\n" if valid_dir is not None else ""
        test_str = f"test: {test_dir.relative_to(save_dir)}\n" if test_dir is not None else ""
        return (
            f"path: {save_dir.absolute()}\n"
            f"{train_str}"
            f"{valid_str}"
            f"{test_str}"
            f"names:\n"
            f"{classes}"
        )
    
    def _visit_image_file(self, image_file): 
        self._w,self._h = pv.image_size(image_file)
        self._image_file = image_file
    
    def _visit_bbox_labelled(self, bbox):
        self._labels.append(str(bbox.label.id))
        points = bbox.normalized_points(self._w,self._h).flatten()
        self._bboxes.append(' '.join(map(str, points)))

In [ ]:
import polvo.bbox as pb

In [ ]:
image_file = pv.ImageFile(str(pv.test.SEG_IMAGE))
class_map = {0: 'a', 1: 'b'}
labels = [pv.Label(i, class_map[i]) for i in [0, 1, 1, 0]]
bboxes = [pb.BBox.from_xyxy(20, 20, 64, 280),
          pb.BBox.from_xywh(10, 72, 478, 72),
          pb.BBox.from_relative_xcycwh(.4, .4, .7, .6, 512, 300),
          pb.OBBox.from_clockwise(256,175, 100,140, 80)]

bboxesl = [pb.BBoxLabeled(bbox, label) for bbox, label in pv.safe_zip(bboxes, labels)]

In [ ]:
record = [image_file, *bboxesl]
converter = YOLO()
# converter.convert_record(record)

In [ ]:
converter.convert_dataset(pv.mkdir('tmp/yolo', parents=True, overwrite=True), class_map,
                          train=[record]*2,
                          valid=[record]*3,
                          test=[record]*2,)

In [ ]:
converter.yolo_yaml(class_map, Path('root_dir'), Path('root_dir/train_dir'))

'path: /home/lgvaz/git/polvo/nbs/root_dir\ntrain: train_dir\nnames:\n  0: a\n  1: b'

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()